In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *

from utils_motor_plot import plot_spect_channel, plot_band_power, plot_all_bands

from scipy.signal.windows import dpss
fs = FS/RS

ROOT_SAVE_DIR = f'{REC_DIR}/5_LMP_MT'

In [ ]:
from tqdm import tqdm

def compute_spectrogram(rec_data, spect_f, spect_t, f_idxs, t_idxs, win_size, dpss_tapers, wt):

    nch = rec_data.shape[0]
    n = win_size
    half_n = int(np.ceil(win_size/2))

    spect = np.zeros((nch, len(spect_t), len(spect_f)))
    for ch, ch_data in tqdm(enumerate(rec_data), desc='processing multi-taper'):
        for ibin, t_idx in enumerate(t_idxs):
            windowed_signal = ch_data[t_idx:t_idx + win_size]

            tapered_signal = np.multiply(windowed_signal, dpss_tapers)

            fft_mag = np.fft.fft(tapered_signal, n, axis=1)
            fft_mag = 2*fft_mag[:,:half_n]/np.sqrt(n)  # Normalize by window size.
            fft_mag[:,0] /= 2

            fft_power = np.real(fft_mag)**2 + np.imag(fft_mag)**2
            fft_power_aggregate = fft_power.T@wt # summation across tapers

            spect[ch][ibin] = fft_power_aggregate[f_idxs]
    
    return spect

In [ ]:
""" multi-taper params """
# T_SCALO (s).      scalogram window size
# T_STEP_SCALO (s). window sliding step size
# W_MT_SCALO (Hz).  multi-taper frequency smoothing: [f0 - W , f0 + W]

NW = T_SCALO*W_MT_SCALO # common choices are 2.5, 3, 3.5, 4
K = int(2*NW - 1) # Number of tapers
wt = np.ones(K)/K # apply unity weight
print(f'{NW=}, {K=}')

mt_win_size = round(fs*T_SCALO)
win_incr = round(fs*T_STEP_SCALO)

n = mt_win_size
half_n = int(np.ceil(n/2))
freq = np.fft.fftfreq(n, d = 1/fs)
half_freq = freq[:half_n]
fbin = half_freq[1]

# DPSS tapers
dpss_tapers, dpss_eigen = dpss(n, NW, K, return_ratios=True)

Generate Spectrogram

In [ ]:
# multi-taper spectrogram
# "clip recording "
# boxcar LMP

# beta_fidxs = np.where(np.logical_and(scalo_f > 13, scalo_f < 30))[0]
# lga_fidxs  = np.where(np.logical_and(scalo_f > 30, scalo_f < 70))[0]
# hga_fidxs  = np.where(np.logical_and(scalo_f > 70, scalo_f < 190))[0]

In [ ]:
for session in GOOD_SESSIONS:
    keys = [key for key in SESSION_KEYS if key.startswith(f'{session:003}')]

    for key in keys:
        load_dir = f'{REC_DIR}/4_zscore/{key}'
        if not os.path.exists(load_dir):
            continue
        
        print(f'Processing session {key}..')

        """ save directory """
        save_data_dir = f'{ROOT_SAVE_DIR}/{key}'
        save_img_dir = f'{ROOT_SAVE_DIR}_imgs/{key}'
        if not os.path.exists(save_data_dir):
            os.makedirs(save_data_dir)
        if not os.path.exists(save_img_dir):
            os.makedirs(save_img_dir)        

        """ load data """
        good_channels   = np.load(f'{load_dir}/good_channels_{key}.npy')
        t               = np.load(f'{load_dir}/t_session_{key}.npy')
        rec_data        = np.load(f'{load_dir}/zscored_recording_session_{key}.npy')

        """ multi-taper params """
        # TJ update 4/12/2024. z-scored input is used now..
        # rec_data -= np.mean(rec_data, axis=0)
        
        # spectrogram cut-off frequency
        # down the pipeline, frequency range becomes tighter..
        f_cut = LPF_FREQ + fbin # will save spectrogram up to f_cut Hz. 

        """ define spectrogram range """
        spect_t_idxs = np.arange(0, len(t) - mt_win_size, win_incr)
        spect_f_idxs = np.where(half_freq < f_cut)[0]
        
        spect_t = t[spect_t_idxs + round(mt_win_size/ 2)]
        spect_f = half_freq[spect_f_idxs]
        
        """ compute spectrogram """
        spect = compute_spectrogram(rec_data, spect_f, spect_t, spect_f_idxs,
                                    spect_t_idxs, mt_win_size, dpss_tapers, wt)
        
        """ clip data prior to LMP extraction (Stavistky 2015)"""
        # TJ update 4/12/2024: clipping no longer applied
        # clipped_data = np.copy(rec_data)

        # clip_vmax = LMP_VCLIP*GAIN*MAX_ADC_CODE/FS_ADC
        # clip_vmin = -1*clip_vmax
        # clipped_data[clipped_data > clip_vmax] = clip_vmax
        # clipped_data[clipped_data < clip_vmin] = clip_vmin

        """ LMP extraction """
        # time index
        bc_win_size = round(T_LMP*fs) # bc: boxcar
        lmp_t_idxs = spect_t_idxs + round(mt_win_size/2) - round(bc_win_size/2)
        lmp_t = t[lmp_t_idxs + round(bc_win_size/2)]
        assert np.array_equal(lmp_t, spect_t)

        # LMP
        # cumsum_data = np.cumsum(clipped_data, axis=0)
        cumsum_data = np.cumsum(rec_data, axis=1)
        # lmp_data = cumsum_data[lmp_t_idxs + bc_win_size,:] - cumsum_data[lmp_t_idxs,:]
        lmp_data = cumsum_data[:,lmp_t_idxs + bc_win_size] - cumsum_data[:,lmp_t_idxs]
        lmp_data /= bc_win_size

        """ z-score """
        # TJ update 4/12/2024: z-scoring is now applied after merging all sessions
        # zspect = zscore_spectrogram(spect)
        # norm_spect = normalize_spectrogram(spect)
        # zlmp_data = (lmp_data - np.mean(lmp_data, axis=0))/np.std(lmp_data, axis=0)

        """ find effective spectrogram frequencies to integrate band power """
        # index of all points that fall under the band
        full_beta_fidxs = np.where(np.logical_and(spect_f >= BETA_FREQ0, spect_f < BETA_FREQ1))[0]
        full_lga_fidxs  = np.where(np.logical_and(spect_f >= LGA_FREQ0,  spect_f < LGA_FREQ1))[0]
        full_hga_fidxs  = np.where(np.logical_and(spect_f >= HGA_FREQ0,  spect_f < HGA_FREQ1))[0]
        
        # index of first frequency f0, s.t. (f0 - half bandwidth) >= FREQ0
        beta_idx0 = np.where(np.round(spect_f[full_beta_fidxs]) >= BETA_FREQ0 + W_MT_SCALO)[0][0]
        lga_idx0  = np.where(np.round(spect_f[full_lga_fidxs])  >= LGA_FREQ0 + W_MT_SCALO)[0][0]
        hga_idx0  = np.where(np.round(spect_f[full_hga_fidxs])  >= HGA_FREQ0 + W_MT_SCALO)[0][0]
        # partition step size
        fidx_incr = int(2*W_MT_SCALO/fbin)
        # effective indices to integrate over
        beta_fidxs = full_beta_fidxs[beta_idx0::fidx_incr]
        lga_fidxs  =  full_lga_fidxs[lga_idx0::fidx_incr]
        hga_fidxs  =  full_hga_fidxs[hga_idx0::fidx_incr]

        """ save """
        np.save(f'{save_data_dir}/good_channels_{key}.npy', good_channels) # carry over
        np.save(f'{save_data_dir}/lmp_{key}.npy', lmp_data)
        np.save(f'{save_data_dir}/spect_{key}.npy', spect) 

        np.save(f'{save_data_dir}/spect_t_{key}.npy', spect_t) 
        np.save(f'{save_data_dir}/spect_f_{key}.npy', spect_f) 
        np.save(f'{save_data_dir}/beta_fidxs_{key}.npy', beta_fidxs)
        np.save(f'{save_data_dir}/lga_fidxs_{key}.npy', lga_fidxs)
        np.save(f'{save_data_dir}/hga_fidxs_{key}.npy', hga_fidxs)

        """ Plot """
        # motion_dir = f'{MOTION_DIR}/{session:03}'
        # motion_t  = np.load(f'{motion_dir}/vel_t_session_{key}.npy')

        # wrist_vel_x   = np.load(f'{motion_dir}/wrist_vel_x_session_{key}.npy')
        # wrist_vel_y   = np.load(f'{motion_dir}/wrist_vel_y_session_{key}.npy')
        # wrist_vel_z   = np.load(f'{motion_dir}/wrist_vel_z_session_{key}.npy')

        # wrist_vel_x = (wrist_vel_x - np.mean(wrist_vel_x))/np.std(wrist_vel_x)
        # wrist_vel_y = (wrist_vel_y - np.mean(wrist_vel_y))/np.std(wrist_vel_y)
        # wrist_vel_z = (wrist_vel_z - np.mean(wrist_vel_z))/np.std(wrist_vel_z)

        """ save channel spectrogram """
        # t0, t1 = spect_t[0], spect_t[-1]
        # f0, f1 = spect_f[0], spect_f[-1]

        # for ch_idx, ch in enumerate(good_channels):
        #     fig, ax = plt.subplots(3, 1, sharex=True)

        #     for ii in range(3):
        #         ax[ii].clear()

        #     plot_spect_channel(ax, t, rec_data, zspect, ch, ch_idx, t0, t1, f0, f1,
        #                        motion_t[:-1], wrist_vel_x, wrist_vel_y, wrist_vel_z, pos_str='Vel.',
        #                        key=key)
        #     plt.savefig(f'{save_img_dir}/channel_{ch}.png', bbox_inches='tight')
        #     plt.close(fig)
        
        """ save band spectrogram """
        # effective paritition band
        # beta_eff_f0 = spect_f[beta_fidxs[0] ] - W_MT_SCALO
        # beta_eff_f1 = spect_f[beta_fidxs[-1]] + W_MT_SCALO
        # lga_eff_f0  = spect_f[lga_fidxs[0] ] - W_MT_SCALO
        # lga_eff_f1  = spect_f[lga_fidxs[-1]] + W_MT_SCALO
        # hga_eff_f0  = spect_f[hga_fidxs[0] ] - W_MT_SCALO
        # hga_eff_f1  = spect_f[hga_fidxs[-1]] + W_MT_SCALO
        # spect_lfs =  spect[:,:,0]
        # spect_beta = np.sum(spect[:,:,beta_fidxs], axis=2)
        # spect_lga  = np.sum(spect[:,:, lga_fidxs], axis=2)
        # spect_hga  = np.sum(spect[:,:, hga_fidxs], axis=2)

        # band_datas = [spect_lfs, spect_beta, spect_lga, spect_hga, lmp_data]
        # title_strs = [
        #             f'Session {key}. Low Freq Band (0-{round(W_MT_SCALO)} ) Hz',
        #             f'Session {key}. β Band ({round(beta_eff_f0)}-{round(beta_eff_f1)} ) Hz',
        #             f'Session {key}. Low-γ Band ({round(lga_eff_f0)}-{round(lga_eff_f1)} ) Hz',
        #             f'Session {key}. High-γ Band ({round(hga_eff_f0)}-{round(hga_eff_f1)} ) Hz',
        #             f'Session {key}. LMP ({round(T_LMP/1e-3)} msec Boxcar)'
        # ]

        # save_fns = ['array_LFS', 'array_beta', 'array_lga', 'array_hga', 'array_LMP']
        # cmaps = ['viridis', 'viridis', 'viridis', 'viridis', 'coolwarm']
        # pos_str = 'Velocity (a.u.)'

        # plt.close('all')
        # fig, ax = plt.subplots(2, 1, sharex=True)
        # for idx in range(len(band_datas)):
        #     for ii in range(2):
        #         ax[ii].clear()

        #     band_data = band_datas[idx]
        #     title_str = title_strs[idx]
        #     cmap = cmaps[idx]
        #     save_fn = save_fns[idx]

        #     plot_band_power(ax, band_data, good_channels, t0, t1,
        #                     motion_t, wrist_vel_x, wrist_vel_y, wrist_vel_z,
        #                     pos_str=pos_str, title_str=title_str, q_cut=0.01, motion_yoff=5,
        #                     cmap=cmap)

        #     plt.savefig(f'{save_img_dir}/{save_fn}.png', bbox_inches='tight')
        
        """ plot all band powers """
        # plt.close('all')
        # fig, ax = plt.subplots(5, 1, sharex=True, figsize=(9, 12))
        # plot_all_bands(ax, lmp_data, spect_beta, spect_lga, spect_hga, good_channels, t0, t1,
        #                 motion_t, wrist_vel_x, wrist_vel_y, wrist_vel_z,
        #                 key=key, pos_str=pos_str, q_cut = 0.01, motion_yoff=5)
        # plt.savefig(f'{save_img_dir}/array_all_bands.png', bbox_inches='tight')

In [ ]:
# for key in SESSION_KEYS:
#     # fn = f'beta_fidxs_{key}.npy'
#     # fn = f'lga_fidxs_{key}.npy'
#     # fn = f'hga_fidxs_{key}.npy'

#     # fn = f'good_channels_{key}.npy'
#     # fn = f'lmp_{key}.npy'
#     # fn = f'spect_{key}.npy'
#     # fn = f'spect_t_{key}.npy'
#     # fn = f'spect_f_{key}.npy'

#     dir0 = f'./recording_preprocessed_v3/5_LMP_MT_bak/{key}'
#     dir1 = f'./recording_preprocessed_v3/5_LMP_MT/{key}'

#     X0 = np.load(f'{dir0}/{fn}')
#     X1 = np.load(f'{dir1}/{fn}')
#     assert np.allclose(X0, X1, equal_nan=True)